# Web Scraping with BeautifulSoup
http://www.crummy.com/software/BeautifulSoup/bs4/doc/#

What is web scraping?
- Extracting information from websites (simulates a human copying and pasting)
- Based on finding patterns in website code (usually HTML)

What are best practices for web scraping?
- Scraping too many pages too fast can get your IP address blocked
- Pay attention to the robots exclusion standard (robots.txt)
- Let's look at http://www.imdb.com/robots.txt

What is HTML?
- Code interpreted by a web browser to produce ("render") a web page
- Let's look at example.html
- Tags are opened and closed
- Tags have optional attributes

How to view HTML code:
- To view the entire page: "View Source" or "View Page Source" or "Show Page Source"
- To view a specific part: "Inspect Element"
- Safari users: Safari menu, Preferences, Advanced, Show Develop menu in menu bar
- Let's inspect example.html

### Aquire Your Data

In [228]:
# read the HTML code for a web page and save as a string
with open('../DAT-DC-10/data/example.html', 'rU') as f:
    html = f.read()

### Look at / Explorre Your Data (html)

In [229]:
print html

In [230]:
# convert HTML into a structured Soup object
from bs4 import BeautifulSoup
b = BeautifulSoup(html, 'html.parser')

In [344]:
# print out the object
# print b
# print b.prettify()

#### 'find' method returns the first matching Tag (and everything inside of it)

In [232]:
# b.find(name='body')
b.find(name='h1')


<h1 id="main">DAT10 Class 6</h1>

In [233]:
# Tags allow you to access the 'inside text'
b.find(name='h1').text

u'DAT10 Class 6'

In [234]:
# Tags also allow you to access their attributes
b.find(name='h1')['id']

u'main'

#### 'find_all' method is useful for finding all matching Tags

In [235]:
b.find_all(name='p')    # returns a ResultSet (like a list of Tags)

[<p class="topic" id="api">First, we are covering APIs, which are useful for getting data.</p>,
 <p class="topic" id="scraping">Then, we are covering web scraping, which is a more flexible way to get data.</p>,
 <p class="topic" id="feedback">Finally, I will ask you to fill out yet another feedback form!</p>,
 <p>Here are some helpful API resources:</p>,
 <p>Here are some helpful web scraping resources:</p>]

### Quiz: What is the datatype returned by 'find_all'? What kinds of operations can we do on that datatype?

Hint: ResultSets can be sliced

In [270]:
# len(b.find_all(name='p'))
# b.find_all(name='p')[0]
# b.find_all(name='p')[0].text
# b.find_all(name='p')[0]['id']
# b.find_all(name='body')

<p class="navCategory">\n<a href="/movies-in-theaters/?ref_=nv_tp_inth_1">Movies</a>,\n                <a href="/chart/toptv/?ref_=nv_tp_tv250_2">TV</a><br/>\n                &amp; <a href="/showtimes/?ref_=nv_tp_sh_3">Showtimes</a></p>

In [237]:
# iterate over a ResultSet
results = b.find_all(name='body')
for tag in results:
    print tag.text

In [238]:
# Make a string with each tag.text seperated by a new line character '\n'

'\n'.join(tag.text for tag in b.find_all(name='p'))

u'First, we are covering APIs, which are useful for getting data.\nThen, we are covering web scraping, which is a more flexible way to get data.\nFinally, I will ask you to fill out yet another feedback form!\nHere are some helpful API resources:\nHere are some helpful web scraping resources:'

### Quiz: How would you write the above as a list comprenhension?

### Limit search by Tag attribute

In [239]:
b.find(name='p', attrs={'id':'scraping'})

<p class="topic" id="scraping">Then, we are covering web scraping, which is a more flexible way to get data.</p>

In [240]:
b.find_all(name='p', attrs={'class':'topic'})
# b.find_all(attrs={'class':'topic'})

[<p class="topic" id="api">First, we are covering APIs, which are useful for getting data.</p>,
 <p class="topic" id="scraping">Then, we are covering web scraping, which is a more flexible way to get data.</p>,
 <p class="topic" id="feedback">Finally, I will ask you to fill out yet another feedback form!</p>]

### Limit search to specific sections

In [241]:
b.find_all(name='li')
b.find(name='ul', attrs={'id':'scraping'}).find_all(name='li')

[<li>Web scraping resource 1</li>, <li>Web scraping resource 2</li>]

## In Class Exercise

1) Find the 'h2' tag and then print its text

In [242]:
b.find(name='h2').text

u'Resource List'

2) Find the 'p' tag with an 'id' value of 'feedback' and then print its text


In [243]:
b.find(name='p', attrs={'id':'feedback'}).text

u'Finally, I will ask you to fill out yet another feedback form!'

3) Find the first 'p' tag and then print the value of the 'id' attribute


In [244]:
b.find(name='p')['id']

u'api'

4) Print the text of all four resources

In [245]:
[tag.text for tag in b.find_all(name='li')]

[u'API resource 1',
 u'API resource 2',
 u'Web scraping resource 1',
 u'Web scraping resource 2']

5) Using a list comprehension can you extract the text of only the API resources?

In [246]:
[tag.text for tag in b.find_all(name='li') if 'API' in tag.text]

[u'API resource 1', u'API resource 2']

### Tool: Selector Gadget
http://selectorgadget.com/

## Scraping IMDb

#### First open your browser and look at the website and the html structure

http://www.imdb.com/title/tt0111161/

#### Get the HTML from the Shawshank Redemption page

In [247]:
import requests
r = requests.get('http://www.imdb.com/title/tt0111161/')

#### What is r? What can we do with it?

In [248]:
r

<Response [200]>

#### convert HTML into Soup

In [249]:
b = BeautifulSoup(r.text, 'html.parser')
# print b

In [250]:
# run this code if you have encoding errors
import sys
reload(sys)
sys.setdefaultencoding('utf8')

#### Get the title

In [251]:
b.find('h1').text

u'The Shawshank Redemption\n                   (1994)\n                   \n'

#### Get the Star Rating (as a float)

In [252]:
# get the star rating (as a float)
float(b.find(name='span', attrs={'itemprop':'ratingValue'}).text)


9.3

#### Get the Movie Rating

In [253]:
panel = b.find('meta', attrs={'itemprop':'contentRating'}) # too many
panel.text

u'R\n| \n                        2h 22min\n                    \n|\nCrime, \nDrama\n|\n14 October 1994 (USA)\n\n '

In [290]:
# Using the Omdbapi, request all years of the 1000 movies in the CSV. Answer the questions below.
import pandas as pd
movies = pd.read_csv('../DAT-DC-10/data/imdb_1000.csv')
top_50 = movies.head(50).copy()

import requests

def get_movie_year(title):
    r = requests.get('http://www.omdbapi.com/?t=' + title + '&r=json&type=movie')
    info = r.json()
    if info['Response'] == 'True':
        try:
            return int(info['Year'])
        except:
            'NA'

from time import sleep
years = []
for title in top_50.title:
    years.append(get_movie_year(title))
    sleep(3)

In [291]:
len(years)

50

In [292]:
top_50["Year"]=years
top_50.tail(5)

,star_rating,title,content_rating,genre,duration,actors_list,Year
45,8.5,Terminator 2: Judgment Day,R,Action,137,"[u'Arnold Schwarzenegger', u'Linda Hamilton', ...",1991
46,8.5,Memento,R,Mystery,113,"[u'Guy Pearce', u'Carrie-Anne Moss', u'Joe Pan...",2000
47,8.5,Taare Zameen Par,PG,Drama,165,"[u'Darsheel Safary', u'Aamir Khan', u'Tanay Ch...",NaN
48,8.5,Dr. Strangelove or: How I Learned to Stop Worr...,PG,Comedy,95,"[u'Peter Sellers', u'George C. Scott', u'Sterl...",1964
49,8.5,The Departed,R,Crime,151,"[u'Leonardo DiCaprio', u'Matt Damon', u'Jack N...",2006


In-Class Exercise

Intro Level:

Using the Omdbapi, request all years of the 1000 movies in the CSV. Answer the questions below.

Challege Challenge Level:

Can you scrape the IMDB Top 250 list (http://www.imdb.com/chart/top?ref_=nv_mv_250_6) and return a Data frame with the movide name, rating, year and the unique movie identifier ie('tt0111161')? Use the function above to scrape each of the movie pages.

Questions:
How many of the Top movies are rated 'R'?
What is the average duration of movies with a star_rating above 9?
What is the average duration of movies before 1985 and after?

In [414]:
# Can you scrape the IMDB Top 250 list (http://www.imdb.com/chart/top?ref_=nv_mv_250_6)
# and return a Data frame with the movide name, rating, year and the unique movie identifier 
# ie('tt0111161')? Use the function above to scrape each of the movie pages.
import pandas as pd

r = requests.get('http://www.imdb.com/chart/top?ref_=nv_mv_250_6')
imdb250 = BeautifulSoup(r.text, 'html.parser')

# prepping list form which to pull title and year
List = [item.text for item in imdb250.find_all(name='td', attrs={'class':'titleColumn'})]
newList = [item.split('\n') for item in List]

# title
titles = [item[2][6:] for item in newList]

# year
years = [int(item[3][1:5]) for item in newList]

# rating
ratingList = [item.text for item in imdb250.find_all(name='td', attrs={'class':'ratingColumn imdbRating'})]
newList2 = [item.split('\n') for item in ratingList]
ratings = [float(item[1]) for item in newList2]

# unique identifier
# http://www.crummy.com/software/BeautifulSoup/bs4/doc/
# imdb250.find_all(name='div', attrs={'class':'wlb_ribbon'})

idList = [item.attrs['data-tconst'] for item in imdb250.find_all(name='div', attrs={'class':'wlb_ribbon'})]


imdb_top250 = pd.DataFrame()
imdb_top250['Title']=titles
imdb_top250['Year']=years
imdb_top250['Rating']=ratings
imdb_top250['UniqueID']=idList
imdb_top250.head(5)

# # read the HTML code for a web page and save as a string
# with open('../IMDbTop250.html', 'rU') as f:
#     imdb250 = f.read()

# type(imdb250)

,Title,Year,Rating,UniqueID
0,The Shawshank Redemption,1994,9.2,tt0111161
1,The Godfather,1972,9.2,tt0068646
2,The Godfather: Part II,1974,9.0,tt0071562
3,The Dark Knight,2008,8.9,tt0468569
4,12 Angry Men,1957,8.9,tt0050083


In [442]:
# Questions:
# How many of the Top movies are rated 'R'? 
# What is the average duration of movies with a star_rating above 9? 
# What is the average duration of movies before 1985 and after?


# imdb250 = BeautifulSoup(r.text, 'html.parser')
# imdb250.find(name='meta', attrs={'itemprop':'contentRating'}).attrs['content']

def get_movie_rating(UniqueID):
    r = requests.get('http://www.imdb.com/title/' + str(UniqueID))
    movie = BeautifulSoup(r.text, 'html.parser')
    return movie.find(name='meta', attrs={'itemprop':'contentRating'}).attrs['content']

from time import sleep
contentRating = []
for ID in imdb_top250.UniqueID:
    contentRating.append(get_movie_rating(ID))
    sleep(3)

imdb_top250['Content Rating']=contentRating
imdb_top250.head()


,Title,Year,Rating,UniqueID,Content Rating
0,The Shawshank Redemption,1994,9.2,tt0111161,R
1,The Godfather,1972,9.2,tt0068646,R
2,The Godfather: Part II,1974,9.0,tt0071562,R
3,The Dark Knight,2008,8.9,tt0468569,PG-13
4,12 Angry Men,1957,8.9,tt0050083,NOT RATED
5,Schindler's List,1993,8.9,tt0108052,R
6,Pulp Fiction,1994,8.9,tt0110912,R
7,"The Good, the Bad and the Ugly",1966,8.9,tt0060196,NOT RATED
8,The Lord of the Rings: The Return of the King,2003,8.9,tt0167260,PG-13
9,Fight Club,1999,8.8,tt0137523,R


In [452]:
# Questions:
# How many of the Top movies are rated 'R'? 
imdb_top250.rename(columns={'Content Rating': 'ContentRating'}, inplace=True)
imdb_top250.head(10)


imdb_top250.groupby('ContentRating').count() # 105 movies in TOp 250 are rated R


,Title,Year,Rating,UniqueID
ContentRating,,,,
APPROVED,18,18,18,18
G,13,13,13,13
M,1,1,1,1
NOT RATED,32,32,32,32
PASSED,3,3,3,3
PG,32,32,32,32
PG-13,33,33,33,33
R,105,105,105,105
TV-MA,1,1,1,1


In [470]:
# Questions:
# What is the average duration of movies with a star_rating above 9? 
# What is the average duration of movies before 1985 and after?


def get_movie_duration(UniqueID):
    r = requests.get('http://www.imdb.com/title/' + str(UniqueID))
    movie = BeautifulSoup(r.text, 'html.parser')
    return int(movie.find(name='time', attrs={'itemprop':'duration'}).attrs['datetime'][2:-1])

from time import sleep
duration = []
for ID in imdb_top250.UniqueID:
    duration.append(get_movie_duration(ID))
    sleep(3)

imdb_top250['Duration']=duration
imdb_top250.head()


,Title,Year,Rating,UniqueID,ContentRating,Duration
0,The Shawshank Redemption,1994,9.2,tt0111161,R,142
1,The Godfather,1972,9.2,tt0068646,R,175
2,The Godfather: Part II,1974,9.0,tt0071562,R,202
3,The Dark Knight,2008,8.9,tt0468569,PG-13,152
4,12 Angry Men,1957,8.9,tt0050083,NOT RATED,96


In [472]:
imdb_top250[imdb_top250.Rating > 9].Duration.mean()


158.5

In [480]:
# Questions: 
# What is the average duration of movies before 1985 and after?
imdb_top250[imdb_top250.Year < 1985].Duration.mean()

127.67256637168141

In [481]:
imdb_top250[imdb_top250.Year > 1985].Duration.mean()

127.82962962962964

### Optional Wed Scraping Homework

First, define a function that accepts an IMDb ID and returns a dictionary of
movie information: title, star_rating, description, content_rating, duration.
The function should gather this information by scraping the IMDb website, not
by calling the OMDb API. (This is really just a wrapper of the web scraping
code we wrote above.)

For example, `get_movie_info('tt0111161')` should return:
```
{'content_rating': 'R',
 'description': u'Two imprisoned men bond over a number of years...',
 'duration': 142,
 'star_rating': 9.3,
 'title': u'The Shawshank Redemption'}
 ```

Then, open the file imdb_ids.txt using Python, and write a for loop that builds
a list in which each element is a dictionary of movie information.
Finally, convert that list into a DataFrame.

### Bonus -- Challenge

Can you scrape the IMDB Top 250 list (http://www.imdb.com/chart/top?ref_=nv_mv_250_6) and return a Data frame with the movide name, rating, year and the unique movie identifier ie('tt0111161')?

Use the function above to scrape each of the movie pages.

**Questions:**

How many of the Top movies are rated 'R'?

What is the average duration of movies with a star_rating above 9?

What is the average duration of movies before 1985 and after?

